In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew, norm
from scipy.optimize import minimize
from numba import njit
from typing import Tuple, Optional
from enum import Enum
import qis
from dataclasses import fields, replace, asdict
from datetime import datetime, timedelta
import scipy.stats as ss
import copy
import scipy

from scipy.interpolate import splrep, BSpline
from numba.typed import List

# analytics
import sys
sys.path.insert(0,'../../') # just for jupyter  notebook
from stochvolmodels.pricers.hawkes_jd_pricer import HawkesJDParams, HawkesJDPricer, hawkesjd_chain_pricer, unpack_and_transform_pars_for_measure_change, unpack_pars

import tensorflow.experimental.numpy as tnp
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from stochvolmodels.data.test_option_chain import get_btc_test_chain_data, get_gld_test_chain_data_6m, get_sqqq_test_chain_data, get_spy_test_chain_data
from stochvolmodels.utils.funcs import to_flat_np_array, set_time_grid, timer, set_seed, transform_to_tfcomplex128, transform_from_tfcomplex128_to_np, slice_option_chain
from stochvolmodels.data.option_chain import OptionChain

import os
from stochvolmodels.pricers.core.bsm_pricer import infer_bsm_implied_vol, compute_bsm_price
import warnings
from stochvolmodels.MLE_estimator import hawkes_jd
from stochvolmodels.MLE_estimator import hawkes_jd_weekday

import pickle
warnings.filterwarnings('ignore')

2023-12-14 14:22:58.669739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def slice_option_chain(option_chain, ids_i):
    option_chain = copy.copy(option_chain)
    replacements = dict()
    for field in fields(option_chain):
        field_name = field.name
        option_chain_value = getattr(option_chain, field_name)
        try:
            replacements[field_name] = [option_chain_value[i] for i in ids_i]
        except:
            replacements[field_name] = option_chain_value
            
    return OptionChain(**replacements)

In [3]:
DAYS_PER_YEAR = 365
HOURS_PER_YEAR = 365 * 24
SECONDS_PER_YEAR = 365 * 24 * 60 * 60  # minute, seconds

In [6]:
options_BTC = pd.read_feather('../../../resources/deribit/BTC_freq_H.feather')
print('loaded')

_id = options_BTC.loc[:,'exchange_time'].apply(lambda x: x.hour) == 9
options_BTC = options_BTC.loc[_id]
print('left only 9am snapshots')

options_BTC.loc[:,'datetime'] = pd.to_datetime(options_BTC.loc[:,'exchange_time'])
options_BTC.loc[:,'datetime'] = options_BTC.loc[:,'datetime'].apply(lambda x:x.replace(tzinfo=None))

options_BTC = options_BTC.drop_duplicates(subset=['index', 'contract'], keep='last')
options_BTC.loc[:,'date'] = options_BTC.loc[:,'index'].copy()
print('kept only the last iv snapshot of every contracts')

options_BTC.loc[:,'forward'] = options_BTC.underlying_price
options_BTC.strike = options_BTC.strike.apply(float)
options_BTC.forward = options_BTC.forward.apply(float)

# Daily options expire every day at 08:00 UTC. Weekly options expire on each Friday of each week at 08:00 UTC. Monthly options expire on the last Friday of each calendar month at 08:00 UTC. Quarterly options expire on the last Friday of each calendar quarter at 08:00 UTC.
# https://www.deribit.com/kb/deribit-introduction-policy
# options_BTC.ttm = options_BTC.ttm.apply(lambda x: x.total_seconds()/SECONDS_PER_YEAR)
options_BTC.loc[:,'ids'] = options_BTC.ttm*365
options_BTC.loc[:,'ids'] = options_BTC.loc[:,'ids'].apply(round)

# remove options without bid or ask
options_BTC = options_BTC.loc[options_BTC.ask_iv != 0,:]
options_BTC = options_BTC.loc[options_BTC.ask_iv < 5,:]

options_BTC = options_BTC.loc[options_BTC.bid_iv != 0,:]
options_BTC = options_BTC.loc[options_BTC.bid_iv < 5,:]

# remove large bid-ask spread
options_BTC.loc[:, 'bid_ask_spread'] = (options_BTC.ask_iv - options_BTC.bid_iv)/options_BTC.bid_iv
_id =  options_BTC.bid_ask_spread < 0.2
options_BTC = options_BTC.loc[_id,:]

# remove deep out-of-money options
_id = options_BTC.strike/options_BTC.forward <= 1.2
_id *= options_BTC.strike/options_BTC.forward >= .8
options_BTC = options_BTC.loc[_id]

# Drop duplicated call and put iv at same strike, keep the smaller spread one
options_BTC = options_BTC.sort_values('bid_ask_spread')
options_BTC = options_BTC.drop_duplicates(['strike', 'exchange_time', 'ids'],keep='first')
options_BTC = options_BTC.sort_values(['ttm','strike'])

loaded
left only 9am snapshots
kept only the last iv snapshot of every contracts


In [7]:
options_BTC.exchange_time.min(), options_BTC.exchange_time.max()

(Timestamp('2019-03-30 09:00:00+0000', tz='UTC'),
 Timestamp('2023-10-04 09:00:00+0000', tz='UTC'))

In [8]:
# options_start = '2019-03-30' 
options_start = '2019-05-30' # 1 month options starting day
options_end   = '2023-10-04' 
dates_arr     = [x.strftime('%Y-%m-%d') for x in pd.date_range(options_start, options_end, freq='D', inclusive="both")]

In [9]:
BTC_option_chain_dict = dict()

for date in dates_arr:
    _date = date + ' 09:00:00+00:00'
    chain_of_day = options_BTC.query('exchange_time == @_date')
    Moneyness = chain_of_day.strike/chain_of_day.underlying_price
    ttms = np.unique(chain_of_day.ttm)
    strikes_ttms = []
    forwards_ttms = []

    bid_iv_ttms = []
    ask_iv_ttms = []

    discfactors_ttms = []
    optiontypes_ttms = []

    ids = np.unique(chain_of_day.ids)

    for _ttm in ttms:
        chain_of_day_ttm_slice = chain_of_day.query('ttm == @_ttm')
        chain_of_day_ttm_slice.loc[:,'Moneyness'] = chain_of_day_ttm_slice.strike/chain_of_day_ttm_slice.underlying_price
        
        chain_of_day_ttm_slice = chain_of_day_ttm_slice.drop_duplicates(['Moneyness'], keep='first')
        mean_price = np.mean(chain_of_day_ttm_slice.underlying_price)
        
        strikes_ttms.append(np.array(chain_of_day_ttm_slice.loc[:,'Moneyness'])*mean_price)
        forwards_ttms.append(np.array([mean_price]))
        bid_iv_ttms.append(np.array(chain_of_day_ttm_slice.bid_iv))
        ask_iv_ttms.append(np.array(chain_of_day_ttm_slice.ask_iv))
        discfactors_ttms.append(np.array([1]))
        optiontypes_ttms.append(np.array(chain_of_day_ttm_slice.optiontype, dtype='<U1'))
        
    BTC_option_chain = OptionChain(ids=ids,
                        ttms=ttms,
                        ticker='BTC',
                        forwards=forwards_ttms,
                        strikes_ttms=strikes_ttms,
                        optiontypes_ttms=optiontypes_ttms,
                        discfactors=discfactors_ttms,
                        bid_ivs=bid_iv_ttms,
                        ask_ivs=ask_iv_ttms)
    
    BTC_option_chain_dict[date] = BTC_option_chain

In [11]:
_1d = 1/365
_2d = 2/365
_3d = 3/365

_1w = 7/365
_2w = 14/365
_3w = 21/365

_1m = 30/365
_2m = 60/365
_3m = 90/365
_6m = 180/365
_9m = 270/365
_1y = 1

_15m = _1y + _3m
_18m = _1y + _6m
_21m = _1y + _9m

_ids_list = ['0d', '1d', '2d', '3d',
        '1w', '2w', '3w', 
        '1m', '2m', '3m',
        '6m', '9m', '1y',
        '15m', '18m', '21m']

ttms_list = [0, _1d, _2d, _3d,
             _1w, _2w, _3w,
             _1m, _2m, _3m,
             _6m, _9m, _1y, 
             _15m, _18m, _21m]

ttms_to_ids = dict(zip(ttms_list, _ids_list))
_ids_to_ttms = dict(zip(_ids_list, ttms_list))

def options_ttms_to_ids(options_ttms):
    # options_ttms must be sorted in ascending order in advance
    options_ttms_ids = dict()

    for k in _ids_to_ttms.keys():
        for ttm in options_ttms[options_ttms+0.001 >= _ids_to_ttms[k]]:
            options_ttms_ids[ttm] = k 
            
    return options_ttms_ids.values()

In [12]:
# Reassign ids
for k in BTC_option_chain_dict.keys():
    _ids = options_ttms_to_ids(BTC_option_chain_dict[k].ttms)
    assert len(BTC_option_chain_dict[k].ids) == len(_ids)
    BTC_option_chain_dict[k].ids = np.array(list(_ids))

In [19]:
with open('BTC_option_chain.pickle', 'wb') as f:
    pickle.dump(BTC_option_chain_dict, f)
